In [168]:
import torch.nn.functional as F
from torch.autograd import Variable
import copy
from collections import Counter
import torch 
import numpy as np
import torch.nn as nn
import torchvision

In [86]:
def read_data(path):
    f = open(path, "r")
    lines = f.readlines()
    f.close()
    data = []
    for l in lines:
        labelSplit = l.replace('\n','').split(' ', 1)
        data.append([labelSplit[0], [word.lower() for word in labelSplit[1].split()]])
    return data

data = read_data('./questions.txt')

In [87]:
def remove_stop_words(data, path):
    stop_words = []
    with open(path) as f:
        stop_words = [word for line in f for word in line.split(",")]
    data_without_stop_words = []
    for k, v in data:
        words = [t for t in v if t not in stop_words]
        data_without_stop_words.append((k, words))
    return data_without_stop_words

data = remove_stop_words(data, './stop_words.txt')

In [88]:
def get_labels(data):
    _labels = []
    for k,v in data:
        _labels.append(k)   
    _unique_label = list(set(_labels))
    _unique_label_dict = {}
    for k,v in enumerate(_unique_label):
        _unique_label_dict[v] = k
    return _unique_label_dict

labels = get_labels(data)

In [89]:
def append_labels(data, labels):        
    cleaned_data = []
    for k,v in data:
        cleaned_data.append((labels[k],v))
        
    return np.array(cleaned_data)

data = append_labels(data, labels)

In [90]:
def create_indexed_vocab(data):
    vocab = []
    for _, sent in data:
        for word in sent:
            vocab.append(word)
    count = Counter(vocab)
    count = {w : count[w] for w in count if count[w] >= 2}
    vocab = []
    for k, v in count.items():
        vocab.append(k)
    indexed_vocab = {word: idx for idx, word in enumerate(vocab)}
    return indexed_vocab
def create_vocab(data):
    total_words_orig = []
    for k,sent in data:
        for word in sent:
            total_words_orig.append(word)
    total_words = list(set(total_words_orig))
    total_words_str = ' '.join(total_words)
    vocab = set(total_words_str.split()) 
    word2idx = {word: idx for idx, word in enumerate(vocab)} # create word index
    return word2idx
word2idx=create_vocab(data)
word2idx['#UNK#'] = len(word2idx)
indexed_vocab = create_indexed_vocab(data)
indexed_vocab['#UNK#'] = len(indexed_vocab)

In [93]:
def load_glove_embeddings(path, indexed_vocab, embedding_dim=300):
    with open(path) as f:
        embeddings = np.zeros((len(indexed_vocab), embedding_dim))
        for line in f.readlines():
            values = line.split()
            word = values[0]
            index = indexed_vocab.get(word)
            if index:
                vector = np.array(values[1:], dtype='float32')
                embeddings[index] = vector
        return torch.from_numpy(embeddings).float()

glove_random = load_glove_embeddings('./glove.small.txt', indexed_vocab)
glove_pre = load_glove_embeddings('./glove.small.txt', word2idx)
embeddings_random = nn.Embedding(glove_random.size(0), glove_random.size(1))
embeddings_pretrained = nn.Embedding.from_pretrained(glove_pre, freeze=True)

In [94]:
def split_train_test(data, test_ratio):
    data_copy = copy.deepcopy(data)
    np.random.shuffle(data_copy)
    test_set_size = int(len(data) * test_ratio)
    test = data_copy[:test_set_size]
    train = data_copy[test_set_size:]
    return train, test

train, test = split_train_test(data, 0.2)
train,dev =split_train_test(train,0.1)

In [95]:
class BOWClassifier(nn.Module):
    def __init__(self,input_size,hidden_size, num_labels):
        super(BOWClassifier, self).__init__()
        self.input_size = input_size
        self.hidden_size  = hidden_size
        self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(self.input_size, num_labels)
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, x):
        hidden = self.fc1(x)
        relu = self.relu(hidden)
        output = self.fc2(x)
        #output = self.sigmoid(output)
        return output

In [101]:
def make_bow_vector(sentence, indexed_vocab,embeddings):
    pt_tensor= torch.zeros(300, dtype=torch.long)
    count = 0
    for word in sentence:
        count += 1
        if word in indexed_vocab:
            pt_tensor = torch.add(pt_tensor, embeddings(torch.LongTensor([indexed_vocab[word]]))[0])
        else:
            pt_tensor = torch.add(pt_tensor, embeddings(torch.LongTensor([indexed_vocab['#UNK#']]))[0])
    pt_tensor=torch.div(pt_tensor, count)
    return pt_tensor

def get_bow_rep(data,word2idx,embeddings):
    bow_data = []
    for label, sent in data:
        bow_data.append(make_bow_vector(sent, word2idx,embeddings))
    return torch.stack(bow_data)
        
training_set = get_bow_rep(train,word2idx,embeddings_pretrained)
training_set_rand = get_bow_rep(train,indexed_vocab,embeddings_random)

In [102]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size = 300
hidden_size = 100
num_classes = 51
num_epochs = 350
learning_rate = 0.0001
batch_size = 100

model = BOWClassifier(input_size, hidden_size, num_classes).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
#training in batches
for epoch in range(num_epochs):
    permutation = torch.randperm(training_set.size()[0])
    for i in range(0, training_set.size()[0], batch_size):
        optimizer.zero_grad()
        indices = permutation[i:i + batch_size]
        batch_features = training_set[indices]
        batch_features = batch_features.reshape(-1, 300).to(device)
        batch_labels = torch.LongTensor([label for label, sent in train[indices]]).to(device)
        batch_outputs = model(batch_features)
        loss = criterion(batch_outputs, batch_labels)
        loss.backward()
        optimizer.step()
    print(loss.item())
    print(epoch)

3.8577375411987305
0
3.766239881515503
1
3.7015557289123535
2
3.6341633796691895
3
3.5899248123168945
4
3.461988687515259
5
3.5528314113616943
6
3.463732957839966
7
3.2996716499328613
8
3.40735125541687
9
3.2944467067718506
10
3.2035000324249268
11
3.2594258785247803
12
3.093846321105957
13
3.092564105987549
14
3.131394863128662
15
3.1297993659973145
16
2.9859044551849365
17
3.0528383255004883
18
2.8519093990325928
19
2.9279050827026367
20
2.905634641647339
21
2.8870065212249756
22
2.7975986003875732
23
2.788588523864746
24
2.7590572834014893
25
2.6858606338500977
26
2.706007719039917
27
2.9241583347320557
28
2.818058729171753
29
2.7365477085113525
30
2.747945547103882
31
2.7600347995758057
32
2.8307321071624756
33
2.7383768558502197
34
2.7723028659820557
35
2.5529563426971436
36
2.568894386291504
37
2.33998703956604
38
2.6940736770629883
39
2.386039972305298
40
2.563384532928467
41
2.7597479820251465
42
2.4870269298553467
43
2.408236503601074
44
2.4940176010131836
45
2.575275897979736

In [103]:
#testing
test_len =len(test)
correct=0
for label,data in test:
    bow_vec = make_bow_vector(data, word2idx,embeddings_pretrained)
    logprobs = model(bow_vec)
    logprobs = F.softmax(logprobs)
    pred = np.argmax(logprobs.data.numpy())
    if pred==label:
        correct+=1
    print('prediction: {}'.format(pred))
    print('actual: {}'.format(label))
accuracy = correct/test_len
print('accuracy: {}'.format(accuracy))

C:\Users\joshy\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  import sys


prediction: 17
actual: 17
prediction: 39
actual: 1
prediction: 5
actual: 5
prediction: 24
actual: 24
prediction: 12
actual: 12
prediction: 13
actual: 13
prediction: 12
actual: 12
prediction: 1
actual: 1
prediction: 11
actual: 11
prediction: 39
actual: 1
prediction: 4
actual: 4
prediction: 42
actual: 42
prediction: 24
actual: 24
prediction: 32
actual: 44
prediction: 13
actual: 13
prediction: 17
actual: 17
prediction: 37
actual: 37
prediction: 17
actual: 17
prediction: 2
actual: 2
prediction: 32
actual: 32
prediction: 13
actual: 13
prediction: 4
actual: 4
prediction: 17
actual: 24
prediction: 20
actual: 20
prediction: 20
actual: 20
prediction: 12
actual: 1
prediction: 12
actual: 12
prediction: 32
actual: 32
prediction: 17
actual: 17
prediction: 17
actual: 17
prediction: 24
actual: 24
prediction: 10
actual: 10
prediction: 13
actual: 1
prediction: 13
actual: 48
prediction: 11
actual: 11
prediction: 17
actual: 17
prediction: 32
actual: 10
prediction: 45
actual: 45
prediction: 13
actual: 13


actual: 32
prediction: 12
actual: 37
prediction: 2
actual: 2
prediction: 11
actual: 11
prediction: 11
actual: 11
prediction: 37
actual: 37
prediction: 17
actual: 17
prediction: 42
actual: 42
prediction: 17
actual: 17
prediction: 17
actual: 10
prediction: 32
actual: 32
prediction: 20
actual: 20
prediction: 13
actual: 1
prediction: 32
actual: 32
prediction: 17
actual: 17
prediction: 39
actual: 39
prediction: 39
actual: 39
prediction: 32
actual: 32
prediction: 17
actual: 17
prediction: 11
actual: 11
prediction: 13
actual: 12
prediction: 2
actual: 2
prediction: 17
actual: 17
prediction: 48
actual: 41
prediction: 37
actual: 49
prediction: 12
actual: 12
prediction: 17
actual: 17
prediction: 17
actual: 17
prediction: 30
actual: 50
prediction: 12
actual: 12
prediction: 32
actual: 32
prediction: 46
actual: 46
prediction: 17
actual: 17
prediction: 12
actual: 17
prediction: 17
actual: 37
prediction: 39
actual: 39
prediction: 12
actual: 12
prediction: 12
actual: 12
prediction: 28
actual: 28
predic

prediction: 17
actual: 49
prediction: 17
actual: 17
prediction: 48
actual: 48
prediction: 11
actual: 48
prediction: 42
actual: 37
prediction: 46
actual: 46
prediction: 12
actual: 12
prediction: 17
actual: 17
prediction: 13
actual: 43
prediction: 12
actual: 12
prediction: 17
actual: 48
prediction: 17
actual: 17
prediction: 28
actual: 13
prediction: 32
actual: 5
prediction: 17
actual: 17
prediction: 12
actual: 12
prediction: 11
actual: 11
prediction: 42
actual: 17
prediction: 12
actual: 12
prediction: 50
actual: 50
prediction: 11
actual: 11
prediction: 4
actual: 4
prediction: 10
actual: 48
prediction: 11
actual: 11
prediction: 12
actual: 12
prediction: 47
actual: 48
prediction: 42
actual: 42
prediction: 46
actual: 46
prediction: 32
actual: 32
prediction: 13
actual: 13
prediction: 17
actual: 17
prediction: 11
actual: 18
prediction: 17
actual: 17
prediction: 11
actual: 34
prediction: 12
actual: 12
prediction: 17
actual: 17
prediction: 13
actual: 13
prediction: 10
actual: 10
prediction: 10


prediction: 17
actual: 17
prediction: 11
actual: 17
prediction: 13
actual: 13
prediction: 1
actual: 1
prediction: 13
actual: 48
prediction: 17
actual: 17
prediction: 2
actual: 42
prediction: 17
actual: 42
prediction: 12
actual: 39
prediction: 2
actual: 42
prediction: 17
actual: 17
prediction: 39
actual: 48
prediction: 45
actual: 50
prediction: 13
actual: 43
prediction: 36
actual: 36
prediction: 42
actual: 42
prediction: 12
actual: 12
prediction: 11
actual: 11
prediction: 13
actual: 13
prediction: 12
actual: 12
prediction: 32
actual: 32
prediction: 11
actual: 11
prediction: 13
actual: 13
prediction: 10
actual: 10
prediction: 17
actual: 42
prediction: 2
actual: 2
prediction: 46
actual: 46
prediction: 13
actual: 13
prediction: 45
actual: 45
prediction: 17
actual: 17
prediction: 20
actual: 20
prediction: 1
actual: 10
prediction: 32
actual: 32
prediction: 17
actual: 17
prediction: 17
actual: 17
prediction: 17
actual: 17
prediction: 17
actual: 17
prediction: 17
actual: 17
prediction: 1
actua

In [176]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim*2, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [184]:
vocab_size=glove_random.size(0)
embedding_dim=glove_random.size(1)
tagset_size=51
hidden_dim=300
learning_rate=0.0001
ModelLSTM = LSTMTagger(embedding_dim,hidden_dim,vocab_size,tagset_size)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(ModelLSTM.parameters(), lr=learning_rate)
num_epochs=2
for epoch in range(num_epochs):
    for label,sent in train:
        ids=[]
        unk = 3375
        labels=[]
        for word in sent:
            index =indexed_vocab.get(word)
            if index:
                ids.append([indexed_vocab[word]])
            else:
                ids.append([unk])
            labels.append(label)
        batch_labels=torch.LongTensor(labels)
        sentence=torch.tensor(ids).squeeze()
        optimizer.zero_grad()
        batch_outputs = ModelLSTM(sentence)
        loss = criterion(batch_outputs, batch_labels)
        loss.backward()
        optimizer.step()
        print("loss",loss.item())
    print("epoch",epoch)

loss 3.965400218963623
loss 3.9585094451904297
loss 3.9286909103393555
loss 4.014394760131836
loss 3.896763563156128
loss 3.906853199005127
loss 3.9653985500335693
loss 3.9868485927581787
loss 3.9041764736175537
loss 3.934847116470337
loss 3.843012571334839
loss 3.9609620571136475
loss 3.931260585784912
loss 3.8878095149993896
loss 3.9689114093780518
loss 3.909637928009033
loss 3.7615151405334473
loss 3.917518138885498
loss 4.008727073669434
loss 3.8205385208129883
loss 3.919041872024536
loss 3.970818042755127
loss 3.8885574340820312
loss 4.0268049240112305
loss 3.909430980682373
loss 3.9801089763641357
loss 3.716451644897461
loss 3.8184876441955566
loss 3.816877603530884
loss 3.864590644836426
loss 3.934432029724121
loss 4.006331443786621
loss 4.003299236297607
loss 3.736837863922119
loss 3.7220942974090576
loss 3.8066439628601074
loss 3.911085605621338
loss 3.9019954204559326
loss 3.610374927520752
loss 3.914324998855591
loss 3.9298441410064697
loss 3.7221107482910156
loss 3.85054183

loss 1.3457279205322266
loss 4.069364070892334
loss 3.6390907764434814
loss 5.355203151702881
loss 3.307368278503418
loss 3.7175793647766113
loss 5.51602840423584
loss 3.5393660068511963
loss 4.618404865264893
loss 2.7209579944610596
loss 3.1764044761657715
loss 1.8217579126358032
loss 5.222905158996582
loss 4.1745710372924805
loss 2.7463860511779785
loss 3.164646625518799
loss 1.9581894874572754
loss 1.952026128768921
loss 2.9843733310699463
loss 1.8490060567855835
loss 3.049217700958252
loss 3.166560173034668
loss 3.5873963832855225
loss 3.8814761638641357
loss 3.974309206008911
loss 6.216309070587158
loss 2.309176206588745
loss 3.918433904647827
loss 2.927311897277832
loss 4.853549480438232
loss 4.847157001495361
loss 2.7260971069335938
loss 1.4104596376419067
loss 1.8183850049972534
loss 1.7837156057357788
loss 3.840062141418457
loss 3.9594995975494385
loss 3.0863873958587646
loss 1.77963387966156
loss 2.398606300354004
loss 3.9325613975524902
loss 1.4238288402557373
loss 5.1110315

loss 4.732065200805664
loss 4.199560642242432
loss 2.9993896484375
loss 2.342907667160034
loss 2.33154559135437
loss 3.3088185787200928
loss 1.3845140933990479
loss 4.550980091094971
loss 2.195838212966919
loss 4.576391696929932
loss 1.0856722593307495
loss 2.7787094116210938
loss 1.0874502658843994
loss 2.410888195037842
loss 1.8989577293395996
loss 4.46309757232666
loss 1.803337574005127
loss 2.424677610397339
loss 4.777656078338623
loss 0.7656320929527283
loss 1.834444284439087
loss 3.436918020248413
loss 1.9149112701416016
loss 1.979071855545044
loss 5.429466724395752
loss 0.7853471040725708
loss 0.5843104124069214
loss 2.8726906776428223
loss 2.892965793609619
loss 2.0395636558532715
loss 3.3705568313598633
loss 4.037670135498047
loss 3.715928316116333
loss 4.579427242279053
loss 2.494342803955078
loss 1.5437989234924316
loss 0.521311342716217
loss 3.4215099811553955
loss 0.7864552140235901
loss 0.49923571944236755
loss 3.4290411472320557
loss 5.529033184051514
loss 2.027049541473

loss 3.6808712482452393
loss 3.4432594776153564
loss 0.07974725216627121
loss 3.0473289489746094
loss 2.710789442062378
loss 3.7508931159973145
loss 3.830096483230591
loss 2.238996982574463
loss 0.35885581374168396
loss 2.2895255088806152
loss 1.1035988330841064
loss 2.842046022415161
loss 0.26126354932785034
loss 1.3156428337097168
loss 4.9742302894592285
loss 2.4126815795898438
loss 0.9259259700775146
loss 1.9165565967559814
loss 3.006671905517578
loss 3.275810956954956
loss 1.3802660703659058
loss 1.169478178024292
loss 3.6431596279144287
loss 2.404942035675049
loss 4.531286716461182
loss 3.784912586212158
loss 3.2303028106689453
loss 3.2398812770843506
loss 2.9922678470611572
loss 1.7707631587982178
loss 3.2360477447509766
loss 3.7925000190734863
loss 3.6085264682769775
loss 4.91506290435791
loss 1.3072893619537354
loss 3.3203272819519043
loss 2.9181506633758545
loss 7.145068168640137
loss 4.922959804534912
loss 2.2382330894470215
loss 0.2116716355085373
loss 0.9119241833686829
los

loss 3.6830503940582275
loss 3.2531678676605225
loss 0.8047245740890503
loss 1.7226642370224
loss 5.189096927642822
loss 2.813504934310913
loss 5.0215277671813965
loss 1.710884690284729
loss 0.909942626953125
loss 2.9106717109680176
loss 2.8426673412323
loss 2.911133289337158
loss 3.232388973236084
loss 0.6916479468345642
loss 1.6985435485839844
loss 0.13220438361167908
loss 1.6722558736801147
loss 2.7112162113189697
loss 1.1484229564666748
loss 2.9166224002838135
loss 3.148705005645752
loss 0.5243179202079773
loss 2.7070419788360596
loss 2.484262704849243
loss 1.4687069654464722
loss 5.00430154800415
loss 0.9169132113456726
loss 0.03850322961807251
loss 2.8754515647888184
loss 4.735922336578369
loss 0.1576310694217682
loss 1.2507085800170898
loss 0.07035592943429947
loss 3.187124490737915
loss 2.9430997371673584
loss 3.282378911972046
loss 5.606223106384277
loss 1.025184988975525
loss 0.8999829888343811
loss 2.4888148307800293
loss 2.615058422088623
loss 1.108219861984253
loss 2.68826

...

loss 3.6168601512908936
loss 0.5183572769165039
loss 0.2414565086364746
loss 0.6498539447784424
loss 0.34002038836479187
loss 1.245844841003418
loss 3.103466033935547
loss 0.2212243676185608
loss 0.19834290444850922
loss 0.5541244745254517
loss 1.3192486763000488
loss 2.9720520973205566
loss 1.4836803674697876
loss 0.3878769278526306
loss 2.262120008468628
loss 0.10557500272989273
loss 3.611536741256714
loss 2.955094575881958
loss 0.052618030458688736
loss 0.07873471826314926
loss 0.8795809745788574
loss 2.3468804359436035
loss 2.019798755645752
loss 3.8748667240142822
loss 3.1698672771453857
loss 0.022716637700796127
loss 2.242196798324585
loss 0.0939946323633194
loss 0.3094004690647125
loss 1.819122552871704
loss 3.8220620155334473
loss 1.9906418323516846
loss 0.01626173034310341
loss 0.11844898015260696
loss 0.21533097326755524
loss 1.0538491010665894
loss 0.01580803468823433
loss 0.3926818072795868
loss 2.5077290534973145
loss 0.6802124977111816
loss 0.8200939893722534
loss 0.58227

loss 0.13817989826202393
loss 0.13561469316482544
loss 0.22397951781749725
loss 0.8335016965866089
loss 2.517242193222046
loss 0.08922187238931656
loss 1.9034998416900635
loss 1.2476236820220947
loss 3.921947956085205
loss 0.2056644856929779
loss 1.1687920093536377
loss 0.032005712389945984
loss 0.09407012909650803
loss 0.5103870034217834
loss 0.15113423764705658
loss 0.8386161923408508
loss 0.24463367462158203
loss 2.1635959148406982
loss 6.720102310180664
loss 2.875762462615967
loss 6.79606294631958
loss 1.322838306427002
loss 0.572536826133728
loss 2.6445326805114746
loss 2.3926970958709717
loss 1.0999023914337158
loss 0.0751558467745781
loss 0.798143208026886
loss 0.12031636387109756
loss 2.054429769515991
loss 1.594327449798584
loss 0.022127747535705566
loss 0.6818682551383972
loss 0.6816141605377197
loss 0.01803796924650669
loss 2.012894868850708
loss 5.008609771728516
loss 0.3177780508995056
loss 0.20992326736450195
loss 0.0343826487660408
loss 0.45367631316185
loss 0.6987280249

loss 0.2968427240848541
loss 0.0836220234632492
loss 2.036729335784912
loss 0.03513067588210106
loss 0.4076237678527832
loss 2.4244754314422607
loss 0.06590425968170166
loss 3.6269474029541016
loss 3.5864830017089844
loss 3.689818859100342
loss 3.2902262210845947
loss 0.34772583842277527
loss 2.212157726287842
loss 0.058434776961803436
loss 2.2236597537994385
loss 0.09905694425106049
loss 3.049403429031372
loss 0.03369184583425522
loss 0.044723428785800934
loss 2.1243786811828613
loss 0.036936331540346146
loss 0.02894272282719612
loss 2.316873788833618
loss 0.4088362157344818
loss 0.03511255234479904
loss 0.02184172160923481
loss 0.4408467710018158
loss 0.045680783689022064
loss 0.21996288001537323
loss 1.8958662748336792
loss 0.26411595940589905
loss 0.03336543217301369
loss 0.42867571115493774
loss 0.22809767723083496
loss 0.08841395378112793
loss 0.04185157269239426
loss 0.3718419671058655
loss 0.043993279337882996
loss 0.03436415269970894
loss 3.4268903732299805
loss 0.034235056489

loss 0.9321527481079102
loss 0.249953031539917
loss 1.348189353942871
loss 2.236290693283081
loss 5.8808064460754395
loss 0.04775691404938698
loss 2.407717227935791
loss 2.5600571632385254
loss 0.10862213373184204
loss 2.552001714706421
loss 1.077473759651184
loss 3.3914268016815186
loss 0.0818847194314003
loss 1.31241774559021
loss 2.5418567657470703
loss 2.9704394340515137
loss 0.6088854074478149
loss 0.14687316119670868
loss 0.28122949600219727
loss 3.453402280807495
loss 2.5191025733947754
loss 1.3403785228729248
loss 0.5189802050590515
loss 1.21533203125
loss 0.1721089482307434
loss 0.013901670463383198
loss 0.7036497592926025
loss 0.03307393938302994
loss 1.0260372161865234
loss 0.11818827688694
loss 0.02341321110725403
loss 0.01775699481368065
loss 4.8681440353393555
loss 0.059524789452552795
loss 0.17325757443904877
loss 0.8156211376190186
loss 1.653967022895813
loss 0.051921941339969635
loss 1.6488497257232666
loss 0.2058098316192627
loss 3.0019888877868652
loss 1.878550648689

KeyboardInterrupt: 

In [185]:
#testing
test_len =len(test)
with torch.no_grad():
    correct = 0
    for label,sent in test:
        ids=[]
        unk = 3375
        labels=[]
        for word in sent:
            index =indexed_vocab.get(word)
            if index:
                ids.append([indexed_vocab[word]])
            else:
                ids.append([unk])
        sentence=torch.tensor(ids).squeeze()
        batch_outputs = ModelLSTM(sentence)
        _, predicted = torch.max(batch_outputs.data, 1)
        pred=calculate_max(predicted)
        if pred==label:
            correct+=1
        print('prediction: {}'.format(pred))
        print('actual: {}'.format(label))
    accuracy = correct/test_len
    print('accuracy: {}'.format(accuracy))

prediction: 17
actual: 17
prediction: 13
actual: 1
prediction: 42
actual: 5
prediction: 24
actual: 24
prediction: 12
actual: 12
prediction: 13
actual: 13
prediction: 12
actual: 12
prediction: 1
actual: 1
prediction: 11
actual: 11
prediction: 48
actual: 1
prediction: 12
actual: 4
prediction: 48
actual: 42
prediction: 24
actual: 24
prediction: 35
actual: 44
prediction: 13
actual: 13
prediction: 17
actual: 17
prediction: 37
actual: 37
prediction: 17
actual: 17
prediction: 2
actual: 2
prediction: 32
actual: 32
prediction: 13
actual: 13
prediction: 4
actual: 4
prediction: 24
actual: 24
prediction: 20
actual: 20
prediction: 20
actual: 20
prediction: 13
actual: 1
prediction: 12
actual: 12
prediction: 32
actual: 32
prediction: 5
actual: 17
prediction: 17
actual: 17
prediction: 24
actual: 24
prediction: 10
actual: 10
prediction: 5
actual: 1
prediction: 13
actual: 48
prediction: 11
actual: 11
prediction: 17
actual: 17
prediction: 10
actual: 10
prediction: 45
actual: 45
prediction: 13
actual: 13


prediction: 32
actual: 32
prediction: 12
actual: 37
prediction: 2
actual: 2
prediction: 11
actual: 11
prediction: 11
actual: 11
prediction: 37
actual: 37
prediction: 17
actual: 17
prediction: 37
actual: 42
prediction: 17
actual: 17
prediction: 10
actual: 10
prediction: 32
actual: 32
prediction: 20
actual: 20
prediction: 42
actual: 1
prediction: 32
actual: 32
prediction: 17
actual: 17
prediction: 39
actual: 39
prediction: 39
actual: 39
prediction: 32
actual: 32
prediction: 17
actual: 17
prediction: 11
actual: 11
prediction: 13
actual: 12
prediction: 2
actual: 2
prediction: 17
actual: 17
prediction: 48
actual: 41
prediction: 39
actual: 49
prediction: 12
actual: 12
prediction: 48
actual: 17
prediction: 17
actual: 17
prediction: 13
actual: 50
prediction: 12
actual: 12
prediction: 32
actual: 32
prediction: 46
actual: 46
prediction: 17
actual: 17
prediction: 42
actual: 17
prediction: 48
actual: 37
prediction: 39
actual: 39
prediction: 12
actual: 12
prediction: 12
actual: 12
prediction: 2
act

actual: 50
prediction: 5
actual: 49
prediction: 17
actual: 17
prediction: 48
actual: 48
prediction: 17
actual: 48
prediction: 37
actual: 37
prediction: 48
actual: 46
prediction: 12
actual: 12
prediction: 39
actual: 17
prediction: 13
actual: 43
prediction: 12
actual: 12
prediction: 42
actual: 48
prediction: 17
actual: 17
prediction: 28
actual: 13
prediction: 1
actual: 5
prediction: 17
actual: 17
prediction: 12
actual: 12
prediction: 11
actual: 11
prediction: 2
actual: 17
prediction: 12
actual: 12
prediction: 50
actual: 50
prediction: 11
actual: 11
prediction: 4
actual: 4
prediction: 17
actual: 48
prediction: 11
actual: 11
prediction: 12
actual: 12
prediction: 17
actual: 48
prediction: 42
actual: 42
prediction: 42
actual: 46
prediction: 4
actual: 32
prediction: 13
actual: 13
prediction: 17
actual: 17
prediction: 11
actual: 18
prediction: 50
actual: 17
prediction: 12
actual: 34
prediction: 12
actual: 12
prediction: 17
actual: 17
prediction: 13
actual: 13
prediction: 17
actual: 10
predicti

prediction: 17
actual: 17
prediction: 12
actual: 17
prediction: 13
actual: 13
prediction: 17
actual: 1
prediction: 48
actual: 48
prediction: 17
actual: 17
prediction: 42
actual: 42
prediction: 50
actual: 42
prediction: 39
actual: 39
prediction: 2
actual: 42
prediction: 17
actual: 17
prediction: 48
actual: 48
prediction: 50
actual: 50
prediction: 1
actual: 43
prediction: 36
actual: 36
prediction: 42
actual: 42
prediction: 12
actual: 12
prediction: 11
actual: 11
prediction: 13
actual: 13
prediction: 12
actual: 12
prediction: 32
actual: 32
prediction: 11
actual: 11
prediction: 13
actual: 13
prediction: 10
actual: 10
prediction: 42
actual: 42
prediction: 2
actual: 2
prediction: 46
actual: 46
prediction: 13
actual: 13
prediction: 45
actual: 45
prediction: 17
actual: 17
prediction: 20
actual: 20
prediction: 10
actual: 10
prediction: 32
actual: 32
prediction: 42
actual: 17
prediction: 17
actual: 17
prediction: 17
actual: 17
prediction: 17
actual: 17
prediction: 17
actual: 17
prediction: 48
ac

In [181]:
def calculate_max(x):
    m=x.numpy()
    largest=dict()
    for y in m:
        if y in largest:
            largest[y]+=1
        else:
            largest[y]=1
    maxx=0
    label=-1
    for i in largest:
        if(largest[i]>maxx):
            maxx=largest[i]
            label=i
    return label